
SwapGame
---



Execute the next cell if necessary to install required libraries in your python installation

In [ ]:
! python -m pip install --upgrade pip
! pip3 install networkx
! pip3 install numpy
! pip3 install pandas
! pip3 install matplotlib

Required libraries

In [ ]:
import random
import networkx as nx
import numpy as np
import copy
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

Social cost

In [ ]:
# Computes the social cost of a node in a graph that is interested in specific nodes, using the SUM metric if use_sum is True, and using the MAX metric otherwise
# if the node cannot reach a node it is interested in, the cost is infinite
def social_cost_player(graph,interests,node:int,use_sum=True):
    if not use_sum:
        return social_cost_player_max(graph,interests,node)
    cost = 0
    spl = dict(nx.all_pairs_shortest_path_length(graph))
    if node in interests:
        for ele in interests[node]:
            if node in spl:
                if ele in spl[node]:
                    cost += spl[node][ele]
                else:
                    cost += float('inf')
    return cost

# Computes the social cost of a node in a graph that is interested in specific nodes, using the MAX metric
# if the node cannot reach a node it is interested in, the cost is infinite
def social_cost_player_max(graph,interests,node:int):
    cost = 0
    spl = dict(nx.all_pairs_shortest_path_length(graph))
    if node in interests:
        for ele in interests[node]:
            if node in spl:
                if ele in spl[node]:
                    if cost < spl[node][ele]: 
                        cost = spl[node][ele]
                else:
                    cost = float('inf')
    return cost

# Computes the social cost of graph using the SUM metric if use_sum is True, and using the MAX metric otherwise
# the social cost is equal to the sum of distances between nodes and their interests (communication cost),  
def social_cost_graph(graph,interests,use_sum=True):
    if not use_sum:
        return social_cost_graph_max(graph,interests)
    cost = 0
    for node in graph.nodes():
        cost += social_cost_player(graph, interests, node)
    #cost += nx.number_of_edges(graph)
    return cost

# Computes the social cost of graph using the MAX metric
# the social cost is equal to the sum of distances between nodes and their interests (communication cost),  
# plus the number of bought edges (construction cost) 
def social_cost_graph_max(graph,interests):
    cost = 0
    for node in graph.nodes():
        cost += social_cost_player_max(graph, interests, node)
    #cost += nx.number_of_edges(graph)
    return cost


Spanning Tree

In [ ]:
"""
Method to compute the best spanning tree of a host graph based on the social cost, based on node interests
Return the best graph (tree) and its associated cost, stating from a host graph given as parameter
"""
def best_spanning_tree(graph,interests,use_sum=True):
    min_cost = float('inf')
    st_min = None

    # search spanning tree with lowest social cost
    for st in nx.SpanningTreeIterator(graph):
        cost = social_cost_graph(st,interests,use_sum)
        if cost < min_cost:
            st_min = st.copy()
            min_cost = cost
    
    return (st_min,min_cost)

"""
Method to compute the best spanning tree based on the social cost
Return the best graph (tree) and its associated cost, stating from a complete host graph of number_nodes
"""
def best_spanning_tree_complete(number_nodes,interests,use_sum=True):
    return best_spanning_tree(nx.complete_graph(number_nodes),interests,use_sum)

"""
Method to compute all spanning tree from a host graph provided as parameter
Returns a list of graphs that are actually spanning trees
"""
def all_spanning_trees(graph):
     res = [st for st in nx.SpanningTreeIterator(graph)]
     return res

def random_spanning_tree(graph):
    return random.choice(all_spanning_trees(graph))


Graph swap a single edge

In [ ]:
"""
Method to update the graph using networkx 
the edge (node,node_to_remove) is deleted
the edge (node,node_to_add) is added
"""
def update_graph(graph,node,node_to_remove,node_to_add):
    graph.remove_edge(node,node_to_remove)
    graph.add_edge(node,node_to_add)
    return graph

Strategies

In [ ]:
"""
Turns a graph into interests for each node : each node's insterests consists of its outgoing neighbors 
in the input directed graph
"""
def get_interests_from_digraph(graph):
    interests = defaultdict(list)
    for u,v in graph.edges(data=False):
        interests[u].append(v)
    return interests

"""
method to compute all swap strategies possible for the node for this turn
--> strategies contains all possibilities for edge swap, the edge to be removed 
is a current neighbor of of the node, while the edge to be added involves any node 
that is not a current neighbor of the current node, but present in the host graph
A solution is a 2-tuple (node_to_remove, node_to_add) in the strategy of the node
Return a list of 2-tuples
"""
def get_swap_strategies_available_host(graph,host_graph,node):

    # get all nodes
    neighbors_available = set([i for i in host_graph.neighbors(node)])

    # remove current node and his neighbors
    if node in neighbors_available:
        neighbors_available.remove(node)
    for ele in graph.neighbors(node):
        if ele in neighbors_available:
            neighbors_available.remove(ele)

    # get all strategies available by swap
    solutions = list()

    for node_to_add in neighbors_available:
        for node_to_remove in graph.neighbors(node):
            tup = (node_to_remove,node_to_add)
            solutions.append(tup)
    
    return solutions

"""
Method to improve the current strategy (wrt social cost for the node)
Returns None if no strategy can improve the current one for the node
"""
def improve_strategy(graph, host_graph, interests, node, allow_disconnect: bool, use_sum=True):
    current_cost = social_cost_player(graph,interests,node,use_sum)
    minimal_cost = current_cost
    minimal_strategy = None
    #print(current_cost)

    # browse all solutions (tuples)
    for node_to_remove, node_to_add in get_swap_strategies_available_host( graph, host_graph, node ): 
        
        # update graph
        update_graph(graph,node,node_to_remove,node_to_add)

        # check if the graph is disconneted by this solution 
        # --> avoid this solution only we have not the autorisation to deconnect the graph
        if allow_disconnect == False and nx.is_connected(graph) == False:
            update_graph(graph,node,node_to_add,node_to_remove) # get the initial topology
            continue

        tmp_cost = social_cost_player(graph,interests,node,use_sum)# get social cost
        # check if we find a best strategy
        if tmp_cost < minimal_cost : 
            minimal_cost = tmp_cost
            minimal_strategy = (node_to_remove,node_to_add)

        update_graph(graph,node,node_to_add,node_to_remove) # get the initial topology
    return minimal_strategy

Simulation Core Functions

In [ ]:
"""
Method use to let a player improve its social cost if possible
Return true if cost was improved, False otherwise
"""
def play_node(graph,host_graph,interests,node,allow_disconnect,use_sum=True):
    best_strategy = improve_strategy(graph,host_graph,interests,node,allow_disconnect,use_sum)
    if best_strategy is not None:
        node_to_remove = best_strategy[0]
        node_to_add = best_strategy[1]
        update_graph(graph,node,node_to_remove,node_to_add)
        return True
    else:
         return False

"""
Execute one node at random
"""
def run_one_random_node(graph,host_graph,interests,allow_disconnect:bool,use_sum=True):
    node = random.choice(range(graph.number_of_nodes()))
    play_node(graph,host_graph,interests,node,allow_disconnect,use_sum)

"""
Execute all nodes in a random order
"""
def run_one_random_round(graph,host_graph,interests,allow_disconnect:bool,use_sum=True):
    nodes = np.random.permutation(np.array(range(graph.number_of_nodes())))
    for node in nodes:
        play_node(graph,host_graph,interests,node,allow_disconnect,use_sum)

"""
Method to check if there is a convergence in the game
Returns true if the system has converged (no node can improve its cost)
Returns false otherwise
"""
def has_converged(graph,host_graph,interests,allow_disconnect,use_sum=True):
    for node in graph.nodes() : 
        if improve_strategy(graph,host_graph,interests,node,allow_disconnect,use_sum) is not None:
            return False
    return True
    

"""
Method to run a single simulation
takes as input:
- initial topology graph (n nodes)
- host graph (n nodes)
- interests graph (<= n nodes graph)
- limit : maximum number of rounds
- ability to disconnect
Returns:
- a 3-tuple with
    - the final graph (equilibrium graph if the associated number is less than limit)
    - the social cost of the final graph
    - a number
        - a positive (or zero) number if convergence is achieved within limit rounds, this number is the number of rounds to reach convergence
        - a -1 if no convergence is reached before limit rounds are run
"""
def run_once(initial_graph, host_graph, interests_graph, limit:int, allow_disconnect:bool, use_sum=True):

    graph = copy.deepcopy(initial_graph)
    interests = get_interests_from_digraph(interests_graph)

    nb_rounds = 0 
    if has_converged(graph, host_graph, interests, allow_disconnect,use_sum): 
        return (graph,social_cost_graph(graph,interests,use_sum),0)
    else:
        while nb_rounds <= limit:
            run_one_random_node(graph,host_graph,interests,allow_disconnect,use_sum)
            nb_rounds += 1
            if has_converged(graph,host_graph,interests,allow_disconnect,use_sum):
                return (graph,social_cost_graph(graph,interests,use_sum),nb_rounds)
        return (graph,social_cost_graph(graph,interests,use_sum),-1)

"""
This methods runs the simulation how_many times with the same parameters
Returns a list of 3-tuple results
"""
def run_several(initial_graph, host_graph, interests_graph, how_many:int, limit:int, allow_disconnect:bool, use_sum=True):
    res = list()
    for i in range(how_many):
        r = run_once(initial_graph,host_graph,interests_graph,limit,allow_disconnect,use_sum)
        res.append(r)
    return res
        
"""
This method runs one simulation several times and reports:
- the best social cost tree  
- two numpy arrays:
    - costs obtained
    - times obtained
"""
def run_one_simulation(host_graph,interests_graph,nb_initial:int,how_many:int,limit:int,use_sum=True,allow_disconnect=False,save_graphs=False):
    
    print(f"host graph: {host_graph.edges()}")
    if(save_graphs):
        nx.draw(host_graph,with_labels=True)
        plt.savefig("host_graph.png")
        plt.clf()
    
    print(f"interests graph: {interests_graph.edges()}")
    if(save_graphs):
        nx.draw(interests_graph,with_labels=True)
        plt.savefig("interests_graph.png")
        plt.clf()
    
    interests = get_interests_from_digraph(interests_graph)
    
    st = best_spanning_tree(host_graph,interests,use_sum)
    if(save_graphs):
        nx.draw(st[0],with_labels=True)
        plt.savefig("best_spanning_tree.png")
        plt.clf()
    print(f"Best spanning tree cost: {st[1]}")

    costs = list()
    times = list()
    
    for sample in range(nb_initial):
        initial_graph = random_spanning_tree(host_graph)
    
        res = run_several(initial_graph,host_graph,interests_graph,how_many,limit,allow_disconnect,use_sum)
        
        i = 0
        for r in res:
            if(save_graphs):
                nx.draw(r[0],with_labels=True)
                plt.savefig(f"res_{sample}_{i}_graph.png")
                plt.clf()
            costs.append(r[1])
            times.append(r[2])
            #print(f"Output {sample}-{i} cost:{r[1]} time:{r[2]}")
            i += 1

    return (st[1],costs,times)

"""
This method runs one simulation campaign, with a given host graph, varying the probability of existence of each interest
It returns five dataframes:
- the cost of the optimal social cost tree
- the minimum cost of the equilibrium social cost
- the maximum cost of the equilibrium social cost
- the cost of the equilibrium social cost
- the time to reach the equilibrium social cost
"""    
def run_campaign(host_graph,allow_disconnect_p=False,use_sum_p=True):
    how_many = 10 
    nb_initial = 10
    limit = 1000

    df_optimal = pd.DataFrame()
    df_min_cost = pd.DataFrame()
    df_max_cost = pd.DataFrame()
    df_cost = pd.DataFrame()
    df_time = pd.DataFrame()

    for p in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        optimals = list()
        min_costs = list()
        max_costs = list()
        costs = list()
        times = list()
        for i in range(10):
            interests_graph = nx.erdos_renyi_graph(host_graph.number_of_nodes(),p,directed=True)
            res = run_one_simulation(host_graph,interests_graph,nb_initial,how_many,limit,allow_disconnect=allow_disconnect_p,use_sum=use_sum_p)
            optimals.append(res[0])
            min_costs.append(min(res[1]))
            max_costs.append(max(res[1]))
            costs.extend(res[1])
            times.extend(res[2])
        df_optimal[p] = optimals
        df_min_cost[p] = min_costs
        df_max_cost[p] = max_costs
        df_cost[p] = costs
        df_time[p] = times
    return (df_optimal,df_cost,df_time,df_min_cost,df_max_cost)

"""
This method runs one simulation campaign, with the pertersen graph as a host graph, varying the probability of existence of each interest
It returns five dataframes:
- the cost of the optimal social cost tree
- the minimum cost of the equilibrium social cost
- the maximum cost of the equilibrium social cost
- the cost of the equilibrium social cost
- the time to reach the equilibrium social cost
"""    
def run_campaign_petersen(allow_disconnect=False,use_sum=True):
    return run_campaign(nx.petersen_graph(),allow_disconnect,use_sum)

"""
This method runs one simulation campaign, with a complete graph of size nodes as a host graph, varying the probability of existence of each interest
It returns five dataframes:
- the cost of the optimal social cost tree
- the minimum cost of the equilibrium social cost
- the maximum cost of the equilibrium social cost
- the cost of the equilibrium social cost
- the time to reach the equilibrium social cost
"""
def run_campaign_complete(size:int,allow_disconnect=False,use_sum=True):
    return run_campaign(nx.complete_graph(size),allow_disconnect,use_sum)

"""
This method runs one simulation campaign, with a random graph of size nodes as a host graph, where each (undirected) edge has probability p, varying the probability of existence of each interest
It returns five dataframes:
- the cost of the optimal social cost tree
- the minimum cost of the equilibrium social cost
- the maximum cost of the equilibrium social cost
- the cost of the equilibrium social cost
- the time to reach the equilibrium social cost
"""
def run_campaign_random(size:int,p:float,allow_disconnect=False,use_sum=True):
    return run_campaign(nx.erdos_renyi_graph(size,p),allow_disconnect,use_sum)

"""
This method runs one simulation campaign, with a ring graph of size nodes as a host graph, varying the probability of existence of each interest
It returns five dataframes:
- the cost of the optimal social cost tree
- the minimum cost of the equilibrium social cost
- the maximum cost of the equilibrium social cost
- the cost of the equilibrium social cost
- the time to reach the equilibrium social cost
"""
def run_campaign_ring(size:int,allow_disconnect=False,use_sum=True):
    return run_campaign(nx.cycle_graph(size),allow_disconnect,use_sum)


__Actually run the simulations__ (from 30 seconds to 20+ hours, the sparser the graph, the faster)

In [ ]:
# SUM metric connected experiments
df_optimal_c_s, df_cost_c_s, df_time_c_s, df_min_cost_c_s, df_max_cost_c_s = run_campaign_ring(10,allow_disconnect=False,use_sum=True)
df_optimal_c_s.to_csv("df_optimal_c_s.csv")
df_min_cost_c_s.to_csv("df_min_cost_c_s.csv")
df_max_cost_c_s.to_csv("df_max_cost_c_s.csv")
df_cost_c_s.to_csv("df_cost_c_s.csv")
df_time_c_s.to_csv("df_time_c_s.csv")

In [ ]:

# SUM metric disconnected experiments
df_optimal_d_s, df_cost_d_s, df_time_d_s, df_min_cost_d_s, df_max_cost_d_s = run_campaign_ring(10,allow_disconnect=True,use_sum=True)
df_optimal_d_s.to_csv("df_optimal_d_s.csv")
df_min_cost_d_s.to_csv("df_min_cost_d_s.csv")
df_max_cost_d_s.to_csv("df_max_cost_d_s.csv")
df_cost_d_s.to_csv("df_cost_d_s.csv")
df_time_d_s.to_csv("df_time_d_s.csv")

In [ ]:

# MAX metric connected experiments
df_optimal_c_m, df_cost_c_m, df_time_c_m, df_min_cost_c_m, df_max_cost_c_m = run_campaign_ring(10,allow_disconnect=False,use_sum=False)
df_optimal_c_m.to_csv("df_optimal_c_m.csv")
df_min_cost_c_m.to_csv("df_min_cost_c_m.csv")
df_max_cost_c_m.to_csv("df_max_cost_c_m.csv")
df_cost_c_m.to_csv("df_cost_c_m.csv")
df_time_c_m.to_csv("df_time_c_m.csv")

In [ ]:

# MAX metric disconnected experiments
df_optimal_d_m, df_cost_d_m, df_time_d_m, df_min_cost_d_m, df_max_cost_d_m = run_campaign_ring(10,allow_disconnect=True,use_sum=False)
df_optimal_d_m.to_csv("df_optimal_d_m.csv")
df_min_cost_d_m.to_csv("df_min_cost_d_m.csv")
df_max_cost_d_m.to_csv("df_max_cost_d_m.csv")
df_cost_d_m.to_csv("df_cost_d_m.csv")
df_time_d_m.to_csv("df_time_d_m.csv")

__Define functions to draw the figures__

In [ ]:
"""
This method produces a figure where the ratio of infinite equilibrium is displayed, given simulation results
"""
def draw_figure_infinite(df_metric, title, metric):
    plt.clf()
    fig, ax = plt.subplots()
    df_plot = pd.DataFrame()
    for p in [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0]:
        df_plot.insert(0,p,1.0-df_metric.groupby(np.isinf(df_metric[p])).count()[1]/1000)
    df_plot.mean(0).plot(label='selfish')
    plt.legend()
    ax.set_xlabel(r'$p$ in $G(n,p)$', fontsize=15)
    ax.set_ylabel(metric, fontsize=15)
    ax.set_title(title)
    ax.grid(True)
    fig.tight_layout()
    plt.show()
    plt.savefig(f"{title}_{metric}.png")

"""
This method produces a figure where the price of anarchy and stability are displayed, given optimal spanning tree, minimun cost simulations, and maximum cost simulations
"""
def draw_figures_price(df_optimal, df_min_cost, df_max_cost, title, metric):
    plt.clf()
    fig, ax = plt.subplots()
    df_price_anarchy = pd.DataFrame(df_optimal / df_max_cost)
    df_price_stability = pd.DataFrame(df_optimal / df_min_cost)
    df_price_anarchy.mean(0).plot(label='Price of Anarchy')
    df_price_stability.mean(0).plot(label="Price of Stability")
    plt.legend()
    ax.set_xlabel(r'$p$ in $G(n,p)$', fontsize=15)
    ax.set_ylabel(metric, fontsize=15)
    ax.set_title(title)
    ax.grid(True)
    fig.tight_layout()
    plt.show()
    plt.savefig(f"{title}_{metric}.png")

"""
This method produces a figure (cost, time) from a single dataset, given optimal spanning tree, cost or time simulation
"""
def draw_figure(df_optimal,df_metric,label_p,title,metric):
    plt.clf()
    fig, ax = plt.subplots()
    df_optimal.mean(0).plot(label='optimal')
    df_metric.mean(0).plot(label=label_p)
    plt.legend()
    ax.set_xlabel(r'$p$ in $G(n,p)$', fontsize=15)
    ax.set_ylabel(metric, fontsize=15)
    ax.set_title(title)
    ax.grid(True)
    fig.tight_layout()
    plt.show()
    plt.savefig(f"{title}_{metric}.png")

"""
This method produces a figure (cost, time) from multiple datasets, given optimal spanning tree, cost or time simulations
"""
def draw_figures(d_df_metric,title,metric):
    plt.clf()
    fig, ax = plt.subplots()
    for l in d_df_metric:
        d_df_metric[l].mean(0).plot(label=l)
    plt.legend()
    ax.set_xlabel(r'$p$ in $G(n,p)$', fontsize=15)
    ax.set_ylabel(metric, fontsize=15)
    ax.set_title(title)
    ax.grid(True)
    fig.tight_layout()
    plt.show()
    plt.savefig(f"{title}_{metric}.png")

Actually draw the figures

In [ ]:
#SUM cost figures (SUM cost simulation must have been run before)

#connected
draw_figure(df_optimal_c_s,df_cost_c_s,'selfish','Social cost: optimal tree vs. selfish connected equilibrium - SUM','cost')
draw_figure_infinite(df_cost_c_s,"Ratio of equilibrium with infinite cost (connectivity) - SUM", "ratio of equilibrium with infinite cost")
draw_figures_price(df_optimal_c_s,df_min_cost_c_s,df_max_cost_c_s,'Price of Anarchy vs. Price of Stability - SUM','Price (connectivity)')

#disconnected
draw_figure(df_optimal_d_s,df_cost_d_s,'selfish','Social cost: optimal tree vs. selfish disconnected equilibrium - SUM','cost')
draw_figure_infinite(df_cost_d_s,"Ratio of equilibrium with infinite cost (disconnection) - SUM", "ratio of equilibrium with infinite cost")
draw_figures_price(df_optimal_c_s,df_min_cost_d_s,df_max_cost_d_s,'Price of Anarchy vs. Price of Stability - SUM','Price (disconnection)')

#connected and disconnected
draw_figures(dict({'selfish (connectivity)':df_cost_c_s,'selfish (disconnection)':df_cost_d_s}),title='Social cost: impact of disconnection - SUM',metric='cost')
draw_figures(dict({'selfish (connectivity)':df_time_c_s,'selfish (disconnection)':df_time_d_s}),title='Convergence time: impact of disconnection - SUM',metric='steps')


In [ ]:
#MAX cost figures (MAX cost simulation must have been run before)

#connected
draw_figure(df_optimal_c_m,df_cost_c_m,'selfish','Social cost: optimal tree vs. selfish connected equilibrium - MAX','cost')
draw_figure_infinite(df_cost_c_m,"Ratio of equilibrium with infinite cost (connectivity) - MAX", "ratio of equilibrium with infinite cost")
draw_figures_price(df_optimal_c_m,df_min_cost_c_m,df_max_cost_c_m,'Price of Anarchy vs. Price of Stability - MAX','Price (connectivity)')

#disconnected
draw_figure(df_optimal_c_m,df_cost_c_m,'selfish','Social cost: optimal tree vs. selfish connected equilibrium - MAX','cost')
draw_figure_infinite(df_cost_d_m,"Ratio of equilibrium with infinite cost (disconnection) - MAX", "ratio of equilibrium with infinite cost")
draw_figures_price(df_optimal_c_m,df_min_cost_d_m,df_max_cost_d_m,'Price of Anarchy vs. Price of Stability - MAX','Price (disconnection)')

#connected and disconnected
draw_figures(dict({'selfish (connectivity)':df_cost_c_m,'selfish (disconnection)':df_cost_d_m}),'Social cost: impact of disconnection - MAX','cost')
draw_figures(dict({'selfish (connectivity)':df_time_c_s,'selfish (disconnection)':df_time_d_m}),'Convergence time: impact of disconnection - MAX','steps')